# Ref-B
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* For a 2-Up portrait system with a GCR of 0.4 the pitch is 2*5.96m = 11.92m
* 2-up Portrait: 5 rows, each with 25 x 2 = 50 modules (Figure 4)
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-B-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-B-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+246.gf6f42f5.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-B-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 11.92 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 50 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-B-1
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-03-22_0000', endtime='2022-03-24_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~22 skyfiles. 
Created 22 skyfiles in /skies/


{'2022-03-22_0730': {'surf_azm': 90.0,
  'surf_tilt': 53.28,
  'theta': -53.28,
  'dni': 0,
  'ghi': 65,
  'dhi': 62,
  'temp_air': 2.4,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0730.rad'},
 '2022-03-22_0830': {'surf_azm': 90.0,
  'surf_tilt': 53.44,
  'theta': -53.44,
  'dni': 2,
  'ghi': 182,
  'dhi': 180,
  'temp_air': 3.0,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0830.rad'},
 '2022-03-22_0930': {'surf_azm': 90.0,
  'surf_tilt': 38.35,
  'theta': -38.35,
  'dni': 3,
  'ghi': 279,
  'dhi': 276,
  'temp_air': 3.9,
  'wind_speed': 2.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0930.rad'},
 '2022-03-22_1030': {'surf_azm': 90.0,
  'surf_tilt': 21.72,
  'theta': -21.72,
  'dni': 172,
  'ghi': 504,
  'dhi': 361,
  'temp_air': 5.0,
  'wind_speed': 3.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_1030.rad'},
 '2022-03-22_1130': {'surf_azm': 90.0,
  'surf_tilt': 3.91,
  'theta': -3.91,
  'dni': 13,
  'ghi': 291,
  'dhi'

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~22 .rad files for gendaylit 1-axis workflow (this takes a minute..)
22 Radfiles created in /objects/

Making 22 octfiles in root directory.
Created 1axis_2022-03-22_0730.oct
Created 1axis_2022-03-22_0830.oct
Created 1axis_2022-03-22_0930.oct
Created 1axis_2022-03-22_1030.oct
Created 1axis_2022-03-22_1130.oct
Created 1axis_2022-03-22_1230.oct
Created 1axis_2022-03-22_1330.oct
Created 1axis_2022-03-22_1430.oct
Created 1axis_2022-03-22_1530.oct
Created 1axis_2022-03-22_1630.oct
Created 1axis_2022-03-22_1730.oct
Created 1axis_2022-03-23_0730.oct
Created 1axis_2022-03-23_0830.oct
Created 1axis_2022-03-23_0930.oct
Created 1axis_2022-03-23_1030.oct
Created 1axis_2022-03-23_1130.oct
Created 1axis_2022-03-23_1230.oct
Created 1axis_2022-03-23_1330.oct
Created 1axis_2022-03-23_1430.oct
Created 1axis_2022-03-23_1530.oct
Created 1axis_2022-03-23_1630.oct
Created 1axis_2022-03-23_1730.oct


In [14]:
trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

Linescan in process: 1axis_2022-03-22_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0730_Row3_Module25.csv
Index: 2022-03-22_0730. Wm2Front: 48.40013333333333. Wm2Back: 14.705525000000002
Linescan in process: 1axis_2022-03-22_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0830_Row3_Module25.csv
Index: 2022-03-22_0830. Wm2Front: 150.54698333333334. Wm2Back: 40.417516666666664
Linescan in process: 1axis_2022-03-22_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0930_Row3_Module25.csv
Index: 2022-03-22_0930. Wm2Front: 255.64305000000002. Wm2Back: 48.702325
Linescan in process: 1axis_2022-03-22_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_1030_Row3_Module25.csv
Index: 2022-03-22_1030. Wm2F

KeyboardInterrupt: 

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-B-Mar-22-M.csv')

In [ ]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-B-Mar-22-S.csv')

In [ ]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-B-Mar-22-N.csv')